In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install opencv-python

EXTRACT IMAGES FROM VIDEOS

In [ ]:
import os
import cv2

def extract_frames(video_path, output_folder, interval=1, duration=4):
    """
    Extract frames from a video for the first `duration` seconds at the given `interval`.

    Args:
    - video_path (str): Path to the video file.
    - output_folder (str): Folder where extracted frames will be saved.
    - interval (int): Extract every `interval`-th frame.
    - duration (int): Number of seconds to extract frames.
    """
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print(f"Error: Couldn't open the video file: {video_path}")
        return False

    # Get the video properties
    frame_rate = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Total number of frames
    max_frame_count = int(frame_rate * duration)  # Max frames to process for `duration` seconds

    print(f"Frame rate: {frame_rate} FPS")
    print(f"Total frames: {total_frames} in {video_path}")
    print(f"Processing first {duration} seconds, equivalent to {max_frame_count} frames.")

    frame_count = 0
    extracted_count = 0

    while frame_count < max_frame_count:
        ret, frame = cap.read()

        if not ret:
            break  # End of video

        # Save the frame at the specified interval
        if frame_count % interval == 0:
            # Use the video filename to organize frames in a subfolder
            video_name = os.path.splitext(os.path.basename(video_path))[0]
            image_filename = os.path.join(output_folder, f"{video_name}_frame_{extracted_count:04d}.jpg")
            cv2.imwrite(image_filename, frame)
            extracted_count += 1
            print(f"Extracted frame {extracted_count} to {image_filename}")

        frame_count += 1

    # Release the video capture object
    cap.release()
    print(f"Extracted {extracted_count} frames from the first {duration} seconds of {video_path}.")
    return True

# Path to the folder containing videos
video_folder = '/content/drive/MyDrive/Gait_Dataset'  # Update with your folder path
output_folder = '/content/drive/MyDrive/extracted_frames/'  # Folder to save extracted frames
interval = 3  # Extract every 3rd frame
duration = 3  # Extract frames for the first 3 seconds

# Get a list of all video files in the folder
video_files = [f for f in os.listdir(video_folder) if f.endswith(('.mp4', '.avi', '.mov'))]

# List to track processed video names
processed_videos = []

# Process each video file
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    success = extract_frames(video_path, output_folder, interval, duration)
    if success:
        processed_videos.append(video_file)

# Print the summary of processed videos
print("\nSummary of processed videos:")
for video in processed_videos:
    print(f"- {video}")


Frame rate: 30.0 FPS
Total frames: 240 in /content/drive/MyDrive/Gait_Dataset/VID_RGB_006.mp4
Processing first 3 seconds, equivalent to 90 frames.
Extracted frame 1 to /content/drive/MyDrive/extracted_frames/VID_RGB_006_frame_0000.jpg
Extracted frame 2 to /content/drive/MyDrive/extracted_frames/VID_RGB_006_frame_0001.jpg
Extracted frame 3 to /content/drive/MyDrive/extracted_frames/VID_RGB_006_frame_0002.jpg
Extracted frame 4 to /content/drive/MyDrive/extracted_frames/VID_RGB_006_frame_0003.jpg
Extracted frame 5 to /content/drive/MyDrive/extracted_frames/VID_RGB_006_frame_0004.jpg
Extracted frame 6 to /content/drive/MyDrive/extracted_frames/VID_RGB_006_frame_0005.jpg
Extracted frame 7 to /content/drive/MyDrive/extracted_frames/VID_RGB_006_frame_0006.jpg
Extracted frame 8 to /content/drive/MyDrive/extracted_frames/VID_RGB_006_frame_0007.jpg
Extracted frame 9 to /content/drive/MyDrive/extracted_frames/VID_RGB_006_frame_0008.jpg
Extracted frame 10 to /content/drive/MyDrive/extracted_frames

ORGANIZE FRAME IMAGES BASED ON THEIR IMAGES

In [ ]:
import os
import shutil

# Define the source directory where the frames are stored
source_dir = "/content/drive/MyDrive/extracted_frames"

# Define the target folders for each emotion
folders = {
    "angry": os.path.join(source_dir, "angry"),
    "happy": os.path.join(source_dir, "happy"),
    "neutral": os.path.join(source_dir, "neutral"),
    "sad": os.path.join(source_dir, "sad"),
}

# Create the folders if they don't exist
for folder in folders.values():
    if not os.path.exists(folder):
        os.makedirs(folder)

# Iterate through the frame pattern
for i in range(84):  # VID_RGB_000 to VID_RGB_083
    frame_prefix = f"VID_RGB_{i:03d}"

    # Define the pattern for the frames within each video (e.g., VID_RGB_000_frame_0000 to VID_RGB_000_frame_0029)
    for j in range(30):  # frame_0000 to frame_0029
        frame_name = f"{frame_prefix}_frame_{j:04d}"

        # Define the target emotion based on the frame index
        if i % 4 == 0:
            emotion = "angry"
        elif i % 4 == 1:
            emotion = "happy"
        elif i % 4 == 2:
            emotion = "neutral"
        elif i % 4 == 3:
            emotion = "sad"

        # Define the source and target paths
        source_path = os.path.join(source_dir, f"{frame_name}.jpg")  # Assuming images are in .jpg format
        target_path = os.path.join(folders[emotion], f"{frame_name}.jpg")

        # Check if the frame exists and move it, otherwise print a message
        if os.path.exists(source_path):
            shutil.move(source_path, target_path)
        else:
            print(f"Warning: {source_path} is missing, skipping.")

print("Frame organization completed.")


Frame organization completed.


*SKIP

In [ ]:
import os
import shutil
import random

# Paths
source_folder = '/content/drive/MyDrive/extracted_frames'
destination_folder = '/content/drive/MyDrive/dataset'

# Subfolder names
emotions = ['angry', 'happy', 'neutral', 'sad']
subfolders = ['train', 'validate', 'test']

# Split ratios
train_ratio = 0.70
validate_ratio = 0.15

# Create the destination structure
for subfolder in subfolders:
    for emotion in emotions:
        os.makedirs(os.path.join(destination_folder, subfolder, emotion), exist_ok=True)

# Function to copy files
def copy_images(image_list, destination_folder):
    for image in image_list:
        src_path = os.path.join(source_folder, emotion, image)
        dest_path = os.path.join(destination_folder, subfolder, emotion, image)

        if not os.path.exists(dest_path):
            shutil.copy(src_path, dest_path)
            print(f"Copied {image} to {dest_path}")
        else:
            print(f"Skipped {image} (already exists)")

# Process each emotion folder
for emotion in emotions:
    emotion_folder = os.path.join(source_folder, emotion)

    # List all image files in the current emotion folder
    image_files = [f for f in os.listdir(emotion_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Shuffle the images
    random.shuffle(image_files)

    # Calculate split sizes
    total_images = len(image_files)
    train_size = int(train_ratio * total_images)
    validate_size = int(validate_ratio * total_images)

    train_images = image_files[:train_size]
    validate_images = image_files[train_size:train_size + validate_size]
    test_images = image_files[train_size + validate_size:]

    # Copy images to respective folders
    for subfolder, image_list in zip(subfolders, [train_images, validate_images, test_images]):
        copy_images(image_list, destination_folder)

print("Dataset preparation complete!")


Copied VID_RGB_056_frame_0008.jpg to /content/drive/MyDrive/dataset/train/angry/VID_RGB_056_frame_0008.jpg
Copied VID_RGB_060_frame_0003.jpg to /content/drive/MyDrive/dataset/train/angry/VID_RGB_060_frame_0003.jpg
Copied VID_RGB_016_frame_0004.jpg to /content/drive/MyDrive/dataset/train/angry/VID_RGB_016_frame_0004.jpg
Copied VID_RGB_000_frame_0004.jpg to /content/drive/MyDrive/dataset/train/angry/VID_RGB_000_frame_0004.jpg
Copied VID_RGB_052_frame_0007.jpg to /content/drive/MyDrive/dataset/train/angry/VID_RGB_052_frame_0007.jpg
Copied VID_RGB_012_frame_0017.jpg to /content/drive/MyDrive/dataset/train/angry/VID_RGB_012_frame_0017.jpg
Copied VID_RGB_060_frame_0006.jpg to /content/drive/MyDrive/dataset/train/angry/VID_RGB_060_frame_0006.jpg
Copied VID_RGB_020_frame_0021.jpg to /content/drive/MyDrive/dataset/train/angry/VID_RGB_020_frame_0021.jpg
Copied VID_RGB_044_frame_0015.jpg to /content/drive/MyDrive/dataset/train/angry/VID_RGB_044_frame_0015.jpg
Copied VID_RGB_024_frame_0019.jpg to 

PATH TO EXTRACTED FRAMES

In [ ]:
import os
import cv2

#path to main dataset folder
dataset_path = '/content/drive/MyDrive/FYP/extracted_frames'

#list all subfolders
subfolders = os.listdir(dataset_path)
print("Subfolders:", subfolders)

Subfolders: ['angry', 'happy', 'neutral', 'sad']


DELETE FOLDER 'DATASET' IF ALREADY EXISTS

In [ ]:
import shutil
import os

# Path to your dataset folder
dataset_path = '/content/drive/MyDrive/FYP/dataset'

# Confirm the folder exists before attempting to delete
if os.path.exists(dataset_path):
    # Remove the entire dataset directory and its contents
    shutil.rmtree(dataset_path)
    print(f"All contents in '{dataset_path}' have been deleted.")
else:
    print(f"Directory '{dataset_path}' does not exist.")

Directory '/content/drive/MyDrive/FYP/dataset' does not exist.


RESIZING (resized folder)

In [ ]:
import numpy as np
import os
import cv2 # Import cv2 for image processing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import Xception, InceptionV3, VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

# Preprocessing images

# Path to save resized images
resized_dataset_path = '/content/drive/MyDrive/FYP/resized_frames'

# Create the destination folder if it doesn't exist
if not os.path.exists(resized_dataset_path):
    os.makedirs(resized_dataset_path)

# Define subfolders and dataset_path
subfolders = ['angry', 'happy', 'neutral', 'sad'] # Define subfolders here
dataset_path = '/content/drive/MyDrive/FYP/extracted_frames'


# Loop through all subfolders
for subfolder in subfolders:
    subfolder_path = os.path.join(dataset_path, subfolder)
    # Create a new subfolder in ResizedImage folder
    resized_subfolder_path = os.path.join(resized_dataset_path, subfolder)

    if not os.path.exists(resized_subfolder_path):
        os.makedirs(resized_subfolder_path)

    # Loop through all files in each subfolder
    for filename in os.listdir(subfolder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')): # Check if it is an image
            img_path = os.path.join(subfolder_path, filename)
            img = cv2.imread(img_path)
            if img is not None: # Check if image loaded successfully
                resized_img = cv2.resize(img, (224, 224))
                resized_img_path = os.path.join(resized_subfolder_path, filename)
                cv2.imwrite(resized_img_path, resized_img)
            else:
                print(f"Could not read image: {img_path}")

print("Resizing complete!")

subfolders = os.listdir(resized_dataset_path)
print("Subfolders:", subfolders)

Resizing complete!
Subfolders: ['angry', 'neutral', 'happy', 'sad']


TRAIN VALIDATE TESTING

In [ ]:
# Organizes image files into a dataset structure for training, validation, and testing
import os
import random
import shutil

# Paths
source_folder = '/content/drive/MyDrive/FYP/resized_frames'
destination_folder = '/content/drive/MyDrive/FYP/dataset'

# Subfolder names
emotions = ['angry', 'happy', 'neutral', 'sad']
subfolders = ['train', 'validate', 'test']

# Split ratios
train_ratio = 0.70
validate_ratio = 0.15

# Create the destination structure
for subfolder in subfolders:
    for emotion in emotions:
        os.makedirs(os.path.join(destination_folder, subfolder, emotion), exist_ok=True)

# Function to copy files
def copy_images(image_list, destination_folder):
    for image in image_list:
        src_path = os.path.join(source_folder, emotion, image)
        dest_path = os.path.join(destination_folder, subfolder, emotion, image)

        if not os.path.exists(dest_path):
            shutil.copy(src_path, dest_path)
            print(f"Copied {image} to {dest_path}")
        else:
            print(f"Skipped {image} (already exists)")

# Process each emotion folder
for emotion in emotions:
    emotion_folder = os.path.join(source_folder, emotion)

    # List all image files in the current emotion folder
    image_files = [f for f in os.listdir(emotion_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Shuffle the images
    random.shuffle(image_files)

    # Calculate split sizes
    total_images = len(image_files)
    train_size = int(train_ratio * total_images)
    validate_size = int(validate_ratio * total_images)

    train_images = image_files[:train_size]
    validate_images = image_files[train_size:train_size + validate_size]
    test_images = image_files[train_size + validate_size:]

    # Copy images to respective folders
    for subfolder, image_list in zip(subfolders, [train_images, validate_images, test_images]):
        copy_images(image_list, destination_folder)

print("Dataset preparation complete!")

Copied VID_RGB_048_frame_0017.jpg to /content/drive/MyDrive/FYP/dataset/train/angry/VID_RGB_048_frame_0017.jpg
Copied VID_RGB_060_frame_0004.jpg to /content/drive/MyDrive/FYP/dataset/train/angry/VID_RGB_060_frame_0004.jpg
Copied VID_RGB_024_frame_0022.jpg to /content/drive/MyDrive/FYP/dataset/train/angry/VID_RGB_024_frame_0022.jpg
Copied VID_RGB_008_frame_0018.jpg to /content/drive/MyDrive/FYP/dataset/train/angry/VID_RGB_008_frame_0018.jpg
Copied VID_RGB_056_frame_0004.jpg to /content/drive/MyDrive/FYP/dataset/train/angry/VID_RGB_056_frame_0004.jpg
Copied VID_RGB_052_frame_0025.jpg to /content/drive/MyDrive/FYP/dataset/train/angry/VID_RGB_052_frame_0025.jpg
Copied VID_RGB_036_frame_0028.jpg to /content/drive/MyDrive/FYP/dataset/train/angry/VID_RGB_036_frame_0028.jpg
Copied VID_RGB_056_frame_0026.jpg to /content/drive/MyDrive/FYP/dataset/train/angry/VID_RGB_056_frame_0026.jpg
Copied VID_RGB_000_frame_0013.jpg to /content/drive/MyDrive/FYP/dataset/train/angry/VID_RGB_000_frame_0013.jpg
C

CHECK

In [ ]:
# Path to the dataset
dataset_path = '/content/drive/MyDrive/FYP/dataset'
subfolders = ['train', 'validate', 'test']
emotions = ['angry', 'happy', 'neutral', 'sad']

# Count and print total images per split
for split in subfolders:
    total = 0
    for emotion in emotions:
        path = os.path.join(dataset_path, split, emotion)
        if os.path.exists(path):
            total += len([f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    print(f"Found {total} images belonging to {len(emotions)} classes.")

Found 1764 images belonging to 4 classes.
Found 376 images belonging to 4 classes.
Found 380 images belonging to 4 classes.
